In [ ]:
# default_exp notify
# hide
_FNAME='notify'

import unittest
from unittest import mock
from nbdev.export import notebook2script
import os
TESTCASE = unittest.TestCase()
_nbpath = os.path.join(_dh[0], _FNAME+'.ipynb')
notebook2script(_nbpath)

Converted notify.ipynb.


# Notify

In [ ]:
#export
import datetime
import logging
logger = logging.getLogger()

import discord

from sewing.mailer import send
from sewing.db import get_guild_emails
import sewing.display

In [ ]:
#export 

from sewing.config import configs

def before(lookback=None):
    now = datetime.datetime.now()
    
    lookback = lookback or configs['lookback']
    return now - datetime.timedelta(**lookback)

class Retriever:
    def __init__(self, after=None):
        self.after = after

    async def get_all_messages_in_channel(self, chan):
        '''
        Returns all messages in all open threads in the channel
        That were sent since `after`
        '''
        messages = {}
        if chan is None:
            return {}
        
        all_threads = [] 
        try:
            all_threads = chan.threads
        except AttributeError:
            logger.info("Channel {} has no threads".format(chan))
        for thread in all_threads:
            logger.info("THREAD {}".format(thread))
            messages[thread] = await self.get_all_messages_in_thread(thread)
        return messages

    async def get_all_messages_in_thread(self, thread):
            messages = await self.retrieve(thread)
            for msg in messages:
                logger.info("{}".format(msg))
            return messages

    async def retrieve(self, messagable, after=None, **kwargs):
        '''
        messagable = thread or channel
        See other kwargs
        https://discordpy.readthedocs.io/en/stable/api.html#discord.TextChannel.history
        '''
        after = after or self.after or before()
        messages = await messagable.history(after=after, **kwargs).flatten()
        return messages


In [ ]:
#export

class FakeResp:
    status_code="didnt send"

def send_content(content:dict):
    logger.info("Sending content: {}".format(content))
    resps = []
    for guild, guild_content in content.items():
        if not guild_content:
            logger.info("{} has no content".format(guild))
            continue
        else:
            logger.info("the content is {}".format(guild_content))
        
        to_emails = get_guild_emails(guild)
        logger.info("sending to {}".format(to_emails))
        if not to_emails or to_emails[0] is None:
            continue
        
        send_this = {guild: guild_content}
        
        html_content = sewing.display.html_content(send_this)
        
        logger.info('{}'.format(html_content))
        
        subject = '[{}] Daily Thread Summary'.format(guild.name)
        
        #resp = FakeResp()
        resp = send(to_emails, subject, html_content)
        resps.append(resp)
        
    logger.info("{}".format(resps))
    return resps
        